In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 --quiet

In [ ]:
import datasets
from datasets import Dataset, DatasetDict
import json
from collections import defaultdict
from sklearn.model_selection import train_test_split
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from datasets import load_dataset
import torch
from peft import LoraConfig, get_peft_model, TaskType
import time
import tqdm
import pandas as pd

import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-ner"

We get the finetune parameters at https://towardsdatascience.com/fine-tune-your-own-llama-2-model-in-a-colab-notebook-df9823a04a32&sca_esv=583722703&strip=1&vwsrc=0

In [ ]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

In [ ]:
type(dataset)

datasets.arrow_dataset.Dataset

In [ ]:
dataset['text'][10]

"<s>[INST] You will create a table with macronutrients, micronutrients and kcal of the following foods: 100g oatmeal, banana 100g, apple 100g, pasta ragù 100g [/INST] Here is a table showing the macronutrients, micronutrients, and kcal of 100g of oatmeal, banana, apple, and pasta ragù:\n\nFood\tCarbohydrates (g)\tFat (g)\tProtein (g)\tFiber (g)\tPotassium (mg)\tVitamin C (mg)\tkcal\nOatmeal\t66.3\t6.9\t16.9\t8.0\t-\t-\t389\nBanana\t23.0\t0.3\t1.3\t2.6\t358\t8.7\t89\nApple\t14.0\t0.2\t0.3\t2.4\t107\t4.6\t52\nPasta Ragù\t53.0\t12.0\t12.0\t2.0\t-\t-\t300\nNote: The above table is an estimation and may vary slightly depending on the specific </s><s>[INST] Thanks, that's awesome! [/INST] No prob, bud! </s>"

In [ ]:
with open("data.json") as f:
  # print(" ".join(f.readlines()))
  dataset = json.loads('\n'.join(f.readlines()))

model_data = []
for row in dataset:
  text = row['text']
  labels2text = defaultdict(set)
  for anno in row['annotations']:
    labels2text[anno['label']].add(anno['entity'])
  for label, outputs in labels2text.items():
    model_data.append({
        'text': f"""<s>[INST] You need to do a named entity extraction task. Extract the {label} from the sentence: {text} Extraction: [/INST] {', '.join(outputs)}</s>""",
        'prompt': f"""You need to do a named entity extraction task. Extract the {label} from the sentence: {text} Extraction:""",
        'answer': f"""{' '.join(outputs)}"""
    })

In [ ]:
test_size = 0.2  # You can adjust this value based on your needs

X_train, X_test = train_test_split(model_data, test_size=test_size, random_state=42)

In [ ]:
dataset = DatasetDict({
    'train': Dataset.from_list(X_train),
    'test':  Dataset.from_list(X_test)
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'prompt', 'answer'],
        num_rows: 67
    })
    test: Dataset({
        features: ['text', 'prompt', 'answer'],
        num_rows: 17
    })
})

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/67 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


In [ ]:
dataset['test'][0]

{'text': '<s>[INST] You need to do a named entity extraction task. Extract the DOMAIN_NAME from the sentence: make\\nCloud Security Service\\nDNS-Layer Network Security\\nSecure Web Gateway\\nSecurity Packages\\n\\nWho we are\\nGlobal Cloud Architecture\\nCloud Network Status\\nCloud Network Activity\\nopendns.com is now Umbrella\\nCisco Umbrella Blog Extraction: [/INST] opendns.com</s>',
 'prompt': '<s>[INST] You need to do a named entity extraction task. Extract the DOMAIN_NAME from the sentence: make\\nCloud Security Service\\nDNS-Layer Network Security\\nSecure Web Gateway\\nSecurity Packages\\n\\nWho we are\\nGlobal Cloud Architecture\\nCloud Network Status\\nCloud Network Activity\\nopendns.com is now Umbrella\\nCisco Umbrella Blog Extraction: [/INST]',
 'answer': 'opendns.com'}

In [ ]:
rst = []
for test_data in tqdm.tqdm(dataset['test']):
  prompt = test_data['prompt']
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
  result = pipe(f"<s>[INST] {prompt} [/INST]")
  peft_model_text_output = result[0]['generated_text']
  ground_truth = test_data['answer']
  rst.append(
      {
          'prompt': prompt,
          'peft_model_text_output': peft_model_text_output,
          'ground_truth': ground_truth
      }
  )

  0%|          | 0/17 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 17/17 [10:02<00:00, 35.45s/it]


In [ ]:
pd.DataFrame.from_records(rst)

,prompt,peft_model_text_output,ground_truth
0,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,opendns.com
1,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,Escola Digital Okoo Playground Sessions Square...
2,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,Oracle
3,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,Dridex
4,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,Colorado Alabama Louisiana
5,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,Sony rootkit technology
6,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,TROJ_DROPPER.HXK
7,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,http://more.1apps[.]com/1.txt
8,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,cdn3[dot]thecritico[dot]com:16122/clickheat/st...
9,<s>[INST] You need to do a named entity extrac...,<s>[INST] <s>[INST] You need to do a named ent...,Backdoor.Win32.Breplibot.b


In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
rst = []
prompts = []
for test_data in tqdm.tqdm(dataset['test']):
  prompt = f"<s>[INST] {test_data['prompt']}. Take a deep breath and analyze the given text. Only output your final extraction in a list. Your output should be concise. Extraction: [/INST]"
  rst.append(
      { #f"""<s>[INST] You need to do a named entity extraction task. Extract the {label} from the sentence: {text} Extraction: [/INST]"""
          'prompt': prompt,
          'ground_truth': test_data['answer']
      }
  )
  prompts.append(prompt)


pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=400)
result = pipe(prompts)

100%|██████████| 17/17 [00:00<00:00, 4320.36it/s]


In [ ]:
for idx, i in enumerate(result):
  rst[idx]['peft_model_text_output'] = i[0]['generated_text'].replace(rst[idx]['prompt'], '')

In [ ]:
pd.DataFrame.from_records(rst)

,prompt,ground_truth,peft_model_text_output
0,<s>[INST] <s>[INST] You need to do a named ent...,opendns.com,"Sure, I can help you with that! Here's the n..."
1,<s>[INST] <s>[INST] You need to do a named ent...,Escola Digital Okoo Playground Sessions Square...,"Sure, here are the named entities extracted ..."
2,<s>[INST] <s>[INST] You need to do a named ent...,Oracle,"Sure, I can help you with that! Here's the n..."
3,<s>[INST] <s>[INST] You need to do a named ent...,Dridex,ACTOR: Dridex</s>
4,<s>[INST] <s>[INST] You need to do a named ent...,Colorado Alabama Louisiana,"Sure, I'd be happy to help you with the name..."
5,<s>[INST] <s>[INST] You need to do a named ent...,Sony rootkit technology,ACTOR: Sony</s>\n</s>
6,<s>[INST] <s>[INST] You need to do a named ent...,TROJ_DROPPER.HXK,"Sure, I can help you with that! Here's the n..."
7,<s>[INST] <s>[INST] You need to do a named ent...,http://more.1apps[.]com/1.txt,"Sure, I can help you with that! Here's the n..."
8,<s>[INST] <s>[INST] You need to do a named ent...,cdn3[dot]thecritico[dot]com:16122/clickheat/st...,"Sure, I can help you with that! Here's the e..."
9,<s>[INST] <s>[INST] You need to do a named ent...,Backdoor.Win32.Breplibot.b,"Sure, I can help you with that! Here's the n..."
